## image

In [6]:
from PIL import Image
import requests
from io import BytesIO


def prepare_image(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    return image

In [24]:
url = 'https://img.youtube.com/vi/ntaO3-n-isc/hqdefault.jpg'
image = prepare_image(url=url)

## vqa

In [1]:
from transformers import AutoProcessor, AutoModelForCausalLM
import torch

/Users/ponienchiang/.pyenv/versions/3.10.9/envs/venv-local-general-ds/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
processor = AutoProcessor.from_pretrained("microsoft/git-base-textvqa")
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-textvqa")

tokenizer_config.json: 100%|██████████| 453/453 [00:00<00:00, 1.98MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 584kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 3.81MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 655kB/s]
config.json: 100%|██████████| 2.82k/2.82k [00:00<00:00, 12.7MB/s]
pytorch_model.bin: 100%|██████████| 709M/709M [04:46<00:00, 2.48MB/s] 
generation_config.json: 100%|██████████| 141/141 [00:00<00:00, 590kB/s]


In [10]:
def answer_img_question(image, q):
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    input_ids = processor(text=q, add_special_tokens=False).input_ids
    input_ids = [processor.tokenizer.cls_token_id] + input_ids
    input_ids = torch.tensor(input_ids).unsqueeze(0)    
    generated_ids = model.generate(pixel_values=pixel_values, input_ids=input_ids, max_length=50)

    return processor.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens=True)

In [20]:
q = 'What does the text say'
answer_img_question(image=image,q=q)

['you use your ipad']

In [25]:
q = 'how many people are in the image?'
answer_img_question(image=image,q=q)

['3']

In [8]:


pixel_values = processor(images=image, return_tensors="pt").pixel_values

question = "what does the front of the bus say at the top?"
input_ids = processor(text=question, add_special_tokens=False).input_ids
input_ids = [processor.tokenizer.cls_token_id] + input_ids
input_ids = torch.tensor(input_ids).unsqueeze(0)

generated_ids = model.generate(pixel_values=pixel_values, input_ids=input_ids, max_length=50)

print("Generated answer:", processor.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens=True))
     

Generated answer: ['unanswerable']
